![](https://7612750.fs1.hubspotusercontent-na1.net/hubfs/7612750/logo%20Senpai_2022%20(1)-1.png)
# ****Proyecto final - Bootcamp Data Science****



## Grupo 2
---



Nombre    | Apellido
--------  |----------
Matilde   | Lanza
Santiago  | Díaz
Cristian  | Lepra
Joaquín   | Domínguez

## Herramientas utilizadas

---
*   Kaggle
*   PowerPoint
*   Neptune.ai


## **Desarrollo de un modelo de computer vision para la deteccion y clasificacion de leucemia linfocitica aguda de tipo B** **texto en negrita**

---



##Marco teórico


La leucemia es un grupo heterogéneo de enfermedades malignas de la médula osea que afectan la produccion de celulas sanguineas de líneas linfoides o mieloides. La leucemia linfocitica aguda (LLA) se da por una acumulacion de mutaciones en celulas progenitoras de la linea linfoide que llevan a una proliferacion de celulas linfoides inmaduras que se liberan al torrente sanguineo. Esta enfermedad provoca una sintomatologia muy diversa que avanza con rapidez.

Dentro de la LLA, se encuentra la de tipo B que afecta los linfocitos B, y la de tipo T que afecta a los linfocitos T.

En la LLA de tipo B se sabe que la maduración de las células leucémicas es un factor importante en el resultado de la terapia. Asimismo, el pronóstico del paciente empeora con el avance del desarrollo de las células B malignas (pre-B temprana, pre-B, pro- B) Boyett et al., (1989).

El diagnóstico definitivo de la leucemia linfoblástica aguda requiere pruebas invasivas, costosas que requieren mucho tiempo. Actualmente el análisis de imágenes de frotis de sangre periférica juega un rol vital en el diagnóstico de la LLA pero puede llevar a errores de diagnóstico debido a la naturaleza inespecífica de la sintomatologia de la enfermedad.

Tambien es frecuente la confusion del diagnostico de la enfermedad con casos benignos en donde se encuentran hematogonias en el frotis. Las hematogonias son precursores normales de linfocitos B que pueden ser detectados en bajan proporcion en sangre periferica en pacientes con varias condiciones clinicas. Estas celulas presentan caracteristicas mnorfologicas similares con los linfoblastos presentes de la leucemia linfoblástica B Boyett et al., (1989).


## Objetivo

---



Desarrollar un modelo predictivo basado en CNN para distinguir casos de Leucemia linfoblástica aguda de tipo B,  de casos negativos sospechosos, a partir de imágenes de frotis de sangre periférica.

## Instalacion e importacion de librerias

---



In [1]:
pip install neptune #Instalación biblioteca Neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 11.2 MB/s eta 0:00:00


In [ ]:
pip install neptune-tensorflow-keras #Instalación biblioteca neptune-tensorflow-keras

In [ ]:
pip install tf-explain #Instalación biblioteca tf-explain

In [ ]:
#Carga de librerias que vamos a usar en el notebook
import numpy as np
import pandas as pd
import cv2 as cv
import seaborn as sns
import os
import matplotlib.pyplot as plt

from PIL import Image
from random import sample

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy, SparseCategoricalAccuracy

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.applications.resnet import ResNet50
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

from tf_explain.core import GradCAM

import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback
from neptune.types import File



# Clases y funciones
---



*Creamos el try/except siguiente ya que kaggle a veces nos da problema con la ruta para cargar el set de datos, es decir, a veces usa esta ruta : '/kaggle/input/Original' y a veces esta otra : '/kaggle/input/leukemia/Original'*

In [ ]:
ruta_directorio_original1 = '/kaggle/input/Original'
ruta_directorio_original2 = '/kaggle/input/leukemia/Original'

try:
    #Intentar acceder al primer directorio
    archivos = os.listdir(ruta_directorio_original1)
    ruta_directorio_original = ruta_directorio_original1
except FileNotFoundError:
    try:
        #Si el primer directorio no existe, intentar acceder al segundo directorio
        archivos = os.listdir(ruta_directorio_original2)
        ruta_directorio_original = ruta_directorio_original2
    except FileNotFoundError:
        print("Ninguno de los directorios existe.")
        #Puedes agregar más lógica aquí según tus necesidades, como salir del programa o asignar un valor por defecto.

#Usar la ruta correcta
print("Ruta correcta:", ruta_directorio_original)

In [ ]:
seed = 42 #Establecemos la semilla en 42

Para que los resultados sean comparables debemos usar siempre el mismo random state por eso fijamos el seed y la fijamos con la funcion siguiente

In [ ]:
tf.keras.utils.set_random_seed(seed) #Establecer semilla para TensorFlow

Vamos a evaluar dos modelos un CNN diseñado por nosotros y una red pre entrenada ResNet50 con fine tuning. Con esta función entrenamos ambos modelos a la vez.

In [ ]:
# Entrenamiento de dos modelos en simultaneo
def train_models(model_cnn, model_resnet, train_generator, val_generator, callbacks):
    # Entrenar el modelo CNN
    history_cnn = model_cnn.fit(train_generator,
                                epochs=50,
                                validation_data=val_generator,
                                callbacks=callbacks)

    # Entrenar el modelo ResNet
    history_resnet = model_resnet.fit(train_generator,
                                      epochs=50,
                                      validation_data=val_generator,
                                      callbacks=callbacks)

    return history_cnn, history_resnet

Esta función gráfica la historia de entrenamiento de un modelo, incluyendo la evolución de la pérdida y el recall. La función recibe un objeto history y un título como argumentos y, para cada uno de ellos, crea un gráfico de pérdida y recall.

In [ ]:
# Evaluacion de modelos en validacion
def plot_model_history(history, title):
    plt.figure(figsize=(12, 5))

    # Gráfica de Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Val Loss', color='cyan')
    plt.title('Loss - ' + title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Gráfica de Recall
    plt.subplot(1, 2, 2)

    # Buscar la métrica de recall en el historial
    recall_metric = None
    for metric in history.history.keys():
        if 'recall' in metric:
            recall_metric = metric
            break

    if recall_metric:
        plt.plot(history.history[recall_metric], label='Train Recall', color='red')
        val_recall_metric = 'val_' + recall_metric
        if val_recall_metric in history.history:
            plt.plot(history.history[val_recall_metric], label='Val Recall', color='orange')

        plt.title('Recall - ' + title)
        plt.xlabel('Epoch')
        plt.ylabel('Recall')
        plt.legend()
    else:
        print("No se encontró la métrica de recall en el historial.")

    plt.show()

#Carga de datos

---

Usamos un dataset de Kaggle que posee imágenes de microscopio, contiene dos carpetas una con las imágenes originales y otra con las imágenes segmentadas, para nuestro análisis utilizamos las imágenes originales. Estas imágenes están divididas en cuatro categorías (Benign, Pro, Pre y Early). Estas categorías son las que decíamos que el modelo prediga.

Dataset:
https://www.kaggle.com/datasets/mehradaria/leukemia

In [ ]:
# Carga de datos
def count_files(folder_names = []):
    for f_name in folder_names:
        count = len(os.listdir(f'{f_name}'))
        print(f'Folder: {f_name} contains {count} images')

Como tenemos las imágenes en carpetas diferentes según su clasificación generamos esta función para contar cuántas imágenes hay por subcarpeta es decir por categoría.

In [ ]:
os.chdir(ruta_directorio_original) # Si no corre el path usar este : '/kaggle/input/leukemia/Original'
print(os.listdir())

count_files(['Benign', 'Early', 'Pre', 'Pro'])

## Montaje de Neptune

---



In [ ]:
#Init Neptune
run = neptune.init_run(
        project="joacodominguez/proyecto-final",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3M2Y4OGJkNC00ZjlmLTQ5MmUtYTg5YS1mMGEzMjEzZmE3Y2QifQ==", # your credentials
)

In [ ]:
model_version = neptune.init_model_version(
    model="PROY-PROY",
    project="joacodominguez/proyecto-final",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3M2Y4OGJkNC00ZjlmLTQ5MmUtYTg5YS1mMGEzMjEzZmE3Y2QifQ==", # your credentials
)

## Exploracion de datos

La función anterior recorre una lista de subcarpetas, cuenta las dimensiones únicas de las imágenes en cada subcarpeta y luego imprime el resultado.

In [ ]:
subcarpetas = ['Benign', 'Early', 'Pre', 'Pro']  # Asumo que esta es la lista de subcarpetas

unique_dimensions = set()

for sub in subcarpetas:
    folder_path = os.path.join(ruta_directorio_original, sub)

    for file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, file)
        with Image.open(image_path) as img:
            unique_dimensions.add(img.size)

if len(unique_dimensions) == 1:
    print(f"\nTodas las imágenes tienen las mismas dimensiones: {unique_dimensions.pop()}")
else:
    print(f"\nSe encontraron {len(unique_dimensions)} dimensiones únicas de imágenes: {unique_dimensions}")

La función anterior realiza varias operaciones para mostrar una cuadrícula de 30 imágenes seleccionadas al azar de las subcarpetas.

In [ ]:
# Inicializa una lista para almacenar nombres de archivos de todas las subcarpetas
nombres_archivos = []

# Itera sobre las subcarpetas y obtén los nombres de archivos
for subcarpeta in os.listdir(ruta_directorio_original):
    ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)
    if os.path.isdir(ruta_subcarpeta):
        nombres_archivos.extend(os.listdir(ruta_subcarpeta))

# Asegúrate de que haya al menos 30 imágenes disponibles
if len(nombres_archivos) >= 30:
    # Selecciona al azar 30 imágenes de la lista
    imagenes_ejemplo = sample(nombres_archivos, 30)

# Configura el diseño de la figura
    filas, columnas = 5, 6
    figura, ejes = plt.subplots(filas, columnas, figsize=(15, 12))
    figura.subplots_adjust(hspace=0.5)

    # Itera sobre las imágenes de ejemplo y muéstralas en la cuadrícula
    for i, nombre_archivo in enumerate(imagenes_ejemplo):
        # Itera sobre las subcarpetas para encontrar la imagen
        for subcarpeta in os.listdir(ruta_directorio_original):
            ruta_subcarpeta = os.path.join(ruta_directorio_original, subcarpeta)
            ruta_imagen = os.path.join(ruta_subcarpeta, nombre_archivo)

            if os.path.isfile(ruta_imagen):
                imagen = cv.imread(ruta_imagen)
                imagen = cv.cvtColor(imagen, cv.COLOR_BGR2RGB)  # Convertir de BGR a RGB para mostrar con matplotlib
                ejes[i // columnas, i % columnas].imshow(imagen)
                ejes[i // columnas, i % columnas].axis('off')
                ejes[i // columnas, i % columnas].set_title(subcarpeta)
                break  # Rompe el bucle una vez que se encuentra la imagen

    plt.show()
else:
    print("No hay suficientes imágenes en las subcarpetas para mostrar 30 ejemplos.")

## MODELADO

---



Este código está creando dos listas, data y labels, que se utilizan para almacenar la información de los archivos de imágenes y sus respectivas etiquetas.

In [ ]:
# Lista de los archivos y los labels
data = [os.path.join(folder,file) for folder in os.listdir(ruta_directorio_original) for file in os.listdir(os.path.join(ruta_directorio_original, folder))]
labels = [folder for folder in os.listdir(ruta_directorio_original) for file in os.listdir(os.path.join(ruta_directorio_original, folder))]

Acá se realiza la división de los datos en conjuntos de entrenamiento, validación y prueba, y luego configura generadores de datos para el aumento de imágenes y la preparación de los datos para el entrenamiento y la evaluación del modelo.

In [ ]:
# train-test split
data_temp,data_test, labels_temp,labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)
data_train, data_val, labels_train, labels_val = train_test_split(data_temp, labels_temp, test_size=0.1, random_state=42)

# data generators para aumentacion en train con rescale
datagen = ImageDataGenerator(rescale=1./255,  # rescale pixel a [0,1]
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': data_train, 'label': labels_train}),
    directory=ruta_directorio_original,
    x_col='filename',
    y_col='label',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': data_val, 'label': labels_val}),
    directory=ruta_directorio_original,
    x_col='filename',
    y_col='label',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

# saco la aumentacion para el test
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': data_test, 'label': labels_test}),
    directory=ruta_directorio_original,
    x_col='filename',
    y_col='label',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)